### Alternative approach using SVM

In [70]:
import gzip
import numpy as np
from sklearn.cross_validation import KFold
from sklearn.cross_validation import StratifiedKFold
from sklearn import svm
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as imgplot
import time
from sklearn.preprocessing import Imputer
from sklearn.metrics import confusion_matrix
%matplotlib inline

In [71]:
start = time.time()
npzfile = np.load('HCS_48x48.npz')
print ("Loaded data in " + str(time.time() - start))
npzfile.files
start = time.time()
cell_rows = npzfile['arr_0']
X = npzfile['arr_1']
Y = npzfile['arr_2']
print ("Loaded data in " + str(time.time() - start))
np.shape(cell_rows), np.shape(Y), np.shape(X), type(X)

Loaded data in 0.000433921813965
Loaded data in 4.88782787323


((52950,), (52950,), (52950, 5, 48, 48), numpy.ndarray)

### Loading the features of the cells

In [72]:
cells_all = pd.read_csv('/home/dueo/data/Genedata/Cells.csv')
cells = cells_all.iloc[cell_rows]
np.shape(cells)

(52950, 407)

In [85]:
cells.ix[0:2,'AreaShape_Area':]
#cells.ix[0:2]

,AreaShape_Area,AreaShape_Center_X,AreaShape_Center_Y,AreaShape_Compactness,AreaShape_Eccentricity,AreaShape_EulerNumber,AreaShape_Extent,AreaShape_FormFactor,AreaShape_MajorAxisLength,AreaShape_MaxFeretDiameter,...,Texture_Variance_ER_3_0,Texture_Variance_ER_5_0,Texture_Variance_Golgi_Actin_3_0,Texture_Variance_Golgi_Actin_5_0,Texture_Variance_Hoechst_3_0,Texture_Variance_Hoechst_5_0,Texture_Variance_Mito_3_0,Texture_Variance_Mito_5_0,Texture_Variance_Nucleoli_3_0,Texture_Variance_Nucleoli_5_0
0,5520,542,43,1.297952,0.837477,1,0.484295,0.141449,118.346136,142.618372,...,0.963408,0.942669,1.432731,1.470543,2.608476,2.653153,1.357492,1.328036,1.516640,1.495042
1,6126,317,52,1.230381,0.769741,1,0.609977,0.375944,116.594822,130.782262,...,1.070573,1.084703,1.917686,1.915602,2.261544,2.322472,1.621720,1.628327,1.489087,1.491358
2,11618,438,82,1.091077,0.522378,1,0.625431,0.284604,136.470704,152.463110,...,0.703105,0.714262,1.621151,1.623899,1.664106,1.705266,1.610848,1.621337,1.679954,1.700747


### Extracting the features and imputing NaNs

In [5]:
X_features = np.asmatrix(cells.ix[:,'AreaShape_Area':])
np.shape(X_features)

(52950, 396)

In [6]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp.fit(X_features)
X_features = imp.transform(X_features)

### Normalization

In [7]:
Xmean = X_features.mean(axis = 0)
XStd = np.sqrt(X_features.var(axis=0))
X = (X_features-Xmean)/(XStd + 0.01)

In [8]:
np.max(X[:,1])

1.8173680921031092

In [9]:
np.min(X), np.max(X), np.mean(X)

(-69.705828219351631, 132.78221451686983, -2.7136455336971616e-18)

In [10]:
Y = np.asarray(Y,dtype='int32')

### Some examples (after normalization)
rows are the different compounds

## Splitting in training and test-set



#### Version 1
80%, 20% randomly choosen

In [11]:
# Currently a Acc. of 0.8250
split = 40000
X_train = X[0:split,:]
Y_train = Y[0:split]
X_test = X[split:,:]
Y_test = Y[split:]

#### Version 3 (Separation of complete well)
Separating a complete well from the training set and use this for testing

In [12]:
idx_test = np.asarray(np.recfromtxt ('test_set_data.csv'))
np.sum(idx_test)

2854

In [13]:
if False:
    X_train = X[idx_test== False]
    X_test = X[idx_test]

    Y_train = Y[idx_test== False]
    Y_test = Y[idx_test]

    X_test.shape, Y_test.shape, X_train.shape, Y_train.shape

#### NO DMSO and  (Separation of complete well)

In [14]:
if False:
    Y = Y -1
    idx_DMSO = np.asarray(np.recfromtxt ('DMSO_data.csv'))
    print('Number of DMSO {}'.format(np.sum(idx_DMSO)))

    idx_test_all = np.asarray(np.recfromtxt ('test_set_data.csv'))

    idx_train = (idx_test == False) & (idx_DMSO == False)
    idx_test  = idx_test & (idx_DMSO == False)

    X_train = X[idx_train]
    Y_train = Y[idx_train]

    X_test = X[idx_test]
    Y_test = Y[idx_test]


    X_test.shape, Y_test.shape, X_train.shape, Y_train.shape #1964 and 10203

In [15]:
# Currently a Acc. of 0.8976
if True:
    np.random.seed(seed=42)
    perm1 = np.random.permutation(len(Y))
    print('Sum of permutation {0}'.format(np.sum(perm1))) #1401824775
    N_split = int(len(Y) * 0.8)
    N_split
    idx_train  = perm1[:N_split]
    idx_test  = perm1[N_split:]


    X_train = X[idx_train,:]
    Y_train = Y[idx_train]
    X_test = X[idx_test,:]
    Y_test = Y[idx_test]

Sum of permutation 1401824775


#### Permuting the training set

In [16]:
perm = np.random.permutation(len(Y_train))
X_train_perm = X_train[perm]
Y_train_perm = Y_train[perm]

### Fisher-LDA

In [87]:
from sklearn.lda import LDA
clf = LDA()
clf.fit(X_train_perm, Y_train_perm)

LDA(n_components=None, priors=None)

In [86]:
Y_train_perm.shape

(42360,)

In [88]:
Y_pred_lda = clf.predict(X_test)

In [89]:
sum(Y_pred_lda == Y_test)/float(len(Y_test))

0.9113314447592068

In [66]:
confusion_matrix(Y_pred_lda, Y_test)

array([[7949,   20,  542,    0],
       [  15,  323,   35,   12],
       [ 251,   60, 1310,    1],
       [   2,    0,    1,   69]])

In [21]:
pred_prob_lda = clf.predict_proba(X_test)

### Random forest

In [22]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_jobs=4,n_estimators=500,oob_score=False)
clf.fit(X_train_perm, Y_train_perm)

RandomForestClassifier(bootstrap=True, compute_importances=None,
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_density=None, min_samples_leaf=1,
            min_samples_split=2, n_estimators=500, n_jobs=4,
            oob_score=False, random_state=None, verbose=0)

In [23]:
Y_pred_rf = clf.predict(X_test)

In [24]:
sum(Y_pred_rf == Y_test)/float(len(Y_test))

0.89150141643059488

In [25]:
pred_prob_RF = clf.predict_proba(X_test)

### 10 Fold X-Validation

We use a 10 fold crossvalidation on the training set to find the optimal parameters

### Sampling 

For the SVM we sample so that the classes have equal proportions.

In [26]:
hist = np.histogram(Y_train_perm, bins=[0, 1, 2, 3,4])
NMAX = np.min(hist[0])
hist, NMAX

((array([32566,  1585,  7877,   332]), array([0, 1, 2, 3, 4])), 332)

In [27]:
idx3 = np.argwhere(Y_train_perm == 3)
idx2 = np.argwhere(Y_train_perm == 2)
idx1 = np.argwhere(Y_train_perm == 1)
idx0 = np.argwhere(Y_train_perm == 0)[0:7877]
idx_d = np.concatenate((idx0,idx1,idx2,idx3))[:,0] #Orderedness is problem in studip k-fold

perm_d = np.random.permutation(len(idx_d))
idx = idx_d[perm_d]
idx.shape

(17671,)

#### In the case of no DSMO

In [28]:
if False:
    hist = np.histogram(YY, bins=[0, 1, 2, 3,4])
    NMAX = np.min(hist[0])
    NMAX = 352
    hist, NMAX

In [29]:
if False:
    idx2 = np.argwhere(YY == 2)[0:NMAX]
    idx1 = np.argwhere(YY == 1)[0:NMAX]
    idx0 = np.argwhere(YY == 0)[0:NMAX]
    idx_d = np.concatenate((idx0,idx1,idx2))[:,0] #Orderedness is problem in studip k-fold
    perm_d = np.random.permutation(len(idx_d))
    idx = idx_d[perm_d]
    idx.shape

In [30]:
XX1 = X_train_perm[idx]
YY1 = Y_train_perm[idx]
np.shape(XX1), np.shape(YY1),np.shape(X_train_perm),np.shape(idx)

((17671, 396), (17671,), (42360, 396), (17671,))

In [31]:
np.histogram(YY1,  bins=[0, 1, 2, 3,4])

(array([7877, 1585, 7877,  332]), array([0, 1, 2, 3, 4]))

In [32]:
YY1[1:10]

array([2, 0, 2, 2, 0, 0, 0, 0, 2], dtype=int32)

In [33]:
def eval_10_Fold(C = 0.5):
    N = len(XX1)
    kf = KFold(N, n_folds=10)
    acc = 0.0
    rounds = 0.0
    for train, test in kf:
        model = svm.SVC(kernel='linear', C=C).fit(XX1[train,],YY1[train])
        res = model.predict(XX1[test])
        acc += sum(res == YY1[test])/float(len(test)) #Accuracy
        rounds += 1.0
        print('round ={0} C={1} acc (cumulated over the folds) ={2}'.format(rounds, C, acc / rounds))
    return acc

In [34]:
#Cs = (1e-3,5e-3,1e-2,5e-2,1e-1,1e0,1e+1,1e2,1e3)
#Cs = (1e-3,5e-3,1e-2,5e-2,1e-1,1e0,1e+1)
#Cs = np.logspace(np.log10(0.001), 2,20)#Schon Komisch, dass man noch ein log drum muss
#res = np.zeros(len(Cs))
#for i,C in enumerate(Cs):
#    res[i] = eval_10_Fold(C)

In [ ]:
plt.semilogx(Cs,res)
#plt.plot(Cs,res)

(Cs,res)

### Evaluation on the test set

Training the model on the whole training data set

In [47]:
#model = svm.SVC(kernel='linear', C=0.01).fit(XX1,YY1)
model = svm.SVC(probability=True, kernel='linear', C=0.01).fit(XX1,YY1)
#model = svm.SVC(probability=True, C=0.01).fit(X_train,Y_train)
#model = svm.SVC(probability=True, C=0.05).fit(XX,YY) #Fit on not sampled

In [48]:
X_test.shape

(10590, 396)

In [46]:
YY1.shape

(17671,)

In [49]:
pred = model.predict(X_test)

In [53]:
model.predict(X_test)
pred_prob_svm = model.predict_proba(X_test)

In [54]:
sum(pred == Y_test)/float(len(Y_test))

0.88092540132200192

In [38]:
pred_prob_RF

array([[ 0.974,  0.   ,  0.026,  0.   ],
       [ 0.862,  0.004,  0.132,  0.002],
       [ 0.966,  0.   ,  0.034,  0.   ],
       ..., 
       [ 0.96 ,  0.   ,  0.04 ,  0.   ],
       [ 0.862,  0.002,  0.132,  0.004],
       [ 0.976,  0.   ,  0.024,  0.   ]])

In [55]:
%load_ext rpy2.ipython
%Rpush pred_prob_RF
%Rpush pred_prob_lda
%Rpush pred_prob_svm
%Rpush Y_test

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


In [56]:
%%R
save(pred_prob_RF, pred_prob_lda, pred_prob_svm, Y_test, file='Test_Like_SIBS_lda_rf_svm.Rdata')

### Calucation of the confusion matrix

In [ ]:
m = confusion_matrix(pred, Y_test)
m

In [ ]:
m.astype('float') / ((m.sum(axis=0)[np.newaxis,:]))

In [ ]:
names_col = ('DMSO True', 'PACLITAXEL True', 'METOCLOPRAMIDE True', 'DIGOXIN True')
m = confusion_matrix(pred, Y_test)
df = pd.DataFrame(m)
df.columns = names_col
names = ('DMSO Pred', 'PACLITAXEL Pred', 'METOCLOPRAMIDE Pred', 'DIGOXIN Pred')
df.index = names
df

In [ ]:
#cm_normalized = m.astype('float') / ((m.sum(axis=1)[:, np.newaxis]))
cm_normalized = m.astype('float') / ((m.sum(axis=0)[np.newaxis,:]))
print('Normalized confusion matrix')


df = pd.DataFrame(cm_normalized)
df.columns = names_col
df.index = names
df

In [ ]:
np.mean(cm_normalized[np.diag_indices(4)])

In [ ]:
1-np.mean(cm_normalized[np.diag_indices(4)])